In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
repChaines="/Users/gilles/pCloud Drive/FOD/GB/Copy/Recherche/Rugby/ChainesReferences/"
fChaines=repChaines+"FR-NZ-fr.xlsx"

dfBrut=pd.read_excel(fChaines, sheet_name="Sheet1")

In [3]:
colSelection=dfBrut.columns[:10]
print ", ".join(colSelection)
dfChaines=dfBrut[colSelection]

Transcription, Cat, Lemme, Modif, Referent, numChaineRef, numChaine, chaineIndex, chaineDep, MentionChaine


In [4]:
dfChaines=dfChaines.drop(columns=["numChaineRef","MentionChaine"])
dfChaines

,Transcription,Cat,Lemme,Modif,Referent,numChaine,chaineIndex,chaineDep
0,bonsoir,INT,bonsoir,NaN,NaN,NaN,NaN,NaN
1,",",PUN,",",NaN,NaN,NaN,NaN,NaN
2,c',PRO:DEM,ce,NaN,NaN,NaN,NaN,NaN
3,était,VER:impf,être,NaN,NaN,NaN,NaN,NaN
4,ceux,PRO:DEM,celui,IND,GTF,1.0,1.0,0.0
5,d',PRP,de,NaN,NaN,NaN,NaN,NaN
6,hier,VER:infi,hier,NaN,NaN,NaN,NaN,NaN
7,.,SENT,.,NaN,NaN,NaN,NaN,NaN
8,voici,ADV,voici,NaN,NaN,NaN,NaN,NaN
9,ceux,PRO:DEM,celui,NC+GTF,GTF,NaN,2.0,1.0


### Préparation des colonnes pour les corrections
Les colonnes visées dans la première partie sont :
- numChaine, chaineIndex, chaineDep  

On convertit toutes les colonnes à Int64 pour éviter le float par défaut.

On crée une copie pour la conservation et des colonnes vides pour le remplacement :
- copier les données à corriger dans oldX
- vider les colonnes originales pour les corrections
- ajouter une colonne pour les appositions

In [5]:
# colonne pour appositions
dfChaines["chaineApp"]=pd.np.nan
# oldX
dfChaines["oldReferent"]=dfChaines.Referent
dfChaines["oldNumChaine"]=dfChaines.numChaine.astype("Int64")
dfChaines["oldChaineIndex"]=dfChaines.chaineIndex#.astype("Int64")
dfChaines["oldChaineDep"]=dfChaines.chaineDep.astype("Int64")

In [6]:
# RAZ colonnes à modifier
dfChaines["numChaine"]=pd.np.nan
dfChaines["chaineIndex"]=pd.np.nan
dfChaines["chaineDep"]=pd.np.nan

In [7]:
dfChaines.head(50)

,Transcription,Cat,Lemme,Modif,Referent,numChaine,chaineIndex,chaineDep,chaineApp,oldReferent,oldNumChaine,oldChaineIndex,oldChaineDep
0,bonsoir,INT,bonsoir,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,",",PUN,",",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,c',PRO:DEM,ce,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,était,VER:impf,être,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ceux,PRO:DEM,celui,IND,GTF,NaN,NaN,NaN,NaN,GTF,1,1.0,0
5,d',PRP,de,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,hier,VER:infi,hier,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,.,SENT,.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,voici,ADV,voici,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,ceux,PRO:DEM,celui,NC+GTF,GTF,NaN,NaN,NaN,NaN,GTF,NaN,2.0,1


- ajouter les nouvelles chaines avant de propager

In [8]:
def addNewChains(row):
    if pd.notnull(row["Modif"]):
        modif=row["Modif"]
        ref=row["Referent"]
        m=re.match(ur"NC(.*)",modif)
        if m:
            tModif=m.group(1)
            print tModif
            print row
    return

In [9]:
dfChaines.apply(addNewChains,axis=1)

+GTF
Transcription        ceux
Cat               PRO:DEM
Lemme               celui
Modif              NC+GTF
Referent              GTF
numChaine             NaN
chaineIndex           NaN
chaineDep             NaN
chaineApp             NaN
oldReferent           GTF
oldNumChaine          NaN
oldChaineIndex          2
oldChaineDep            1
Name: 9, dtype: object
+JNZ7
Transcription        Cole
Cat                   NAM
Lemme                Cole
Modif             NC+JNZ7
Referent              NaN
numChaine             NaN
chaineIndex           NaN
chaineDep             NaN
chaineApp             NaN
oldReferent           NaN
oldNumChaine          NaN
oldChaineIndex        NaN
oldChaineDep          NaN
Name: 1605, dtype: object

Transcription         ils
Cat               PRO:PER
Lemme                  il
Modif                  NC
Referent              GPF
numChaine             NaN
chaineIndex           NaN
chaineDep             NaN
chaineApp             NaN
oldReferent           GPF
old

+GTF
Transcription         les
Cat               DET:ART
Lemme                  le
Modif              NC+GTF
Referent              GTF
numChaine             NaN
chaineIndex           NaN
chaineDep             NaN
chaineApp             NaN
oldReferent           GTF
oldNumChaine          NaN
oldChaineIndex          3
oldChaineDep            0
Name: 21369, dtype: object
+GTF
Transcription     rouges
Cat                  ADJ
Lemme              rouge
Modif             NC+GTF
Referent             GTF
numChaine            NaN
chaineIndex          NaN
chaineDep            NaN
chaineApp            NaN
oldReferent          GTF
oldNumChaine         NaN
oldChaineIndex         3
oldChaineDep           0
Name: 21370, dtype: object


0        None
1        None
2        None
3        None
4        None
5        None
6        None
7        None
8        None
9        None
10       None
11       None
12       None
13       None
14       None
15       None
16       None
17       None
18       None
19       None
20       None
21       None
22       None
23       None
24       None
25       None
26       None
27       None
28       None
29       None
         ... 
21912    None
21913    None
21914    None
21915    None
21916    None
21917    None
21918    None
21919    None
21920    None
21921    None
21922    None
21923    None
21924    None
21925    None
21926    None
21927    None
21928    None
21929    None
21930    None
21931    None
21932    None
21933    None
21934    None
21935    None
21936    None
21937    None
21938    None
21939    None
21940    None
21941    None
Length: 21942, dtype: object

In [ ]:
dfChaines.oldNumChaine=dfChaines[dfChaines.Referent.notnull()].oldNumChaine.ffill()

In [ ]:
def updateRows(row):
    numChaine=np.NaN
    chaineIndex=np.NaN
    chaineDep=np.NaN
    idx=row.index
    if pd.isna(row["Modif"]):
        row["numChaine"]=row["oldNumChaine"]
        row["chaineIndex"]=row["oldChaineIndex"]
        row["chaineDep"]=row["oldChaineDep"]
    else:
        modif=row["Modif"]
        m=re.match(ur"^R\s*\+\s*(\d+)\s*$",modif)
        if row["Modif"]=="A":
            row["numChaine"]=np.NaN
            row["chaineApp"]=row["oldNumChaine"]
        elif row["Modif"]=="D":
            row["Referent"]=np.NaN
        elif m:
            row["numChaine"]=int(m.group(1))            
            row["chaineIndex"]=row["oldChaineIndex"]
            row["chaineDep"]=row["oldChaineDep"]
        else:
            row["numChaine"]=row["oldNumChaine"]
            row["chaineIndex"]=row["oldChaineIndex"]
            row["chaineDep"]=row["oldChaineDep"]            
    return row


In [ ]:
dfResult=dfChaines.apply(updateRows,axis=1)

In [ ]:
dfResult.to_excel(fChaines.replace(".xls","-GB.xls"))